In [1]:
import os
import scanpy as sc
import squidpy as sq
import anndata as an
import pandas as pd

### Распаковка tar.gz файла

In [3]:
import tarfile

# open file
file = tarfile.open('tar_data/macaque.tar.gz')
# extracting file
file.extractall('data/spatial_data/')
file.close()

## Объединим файлы с разметкой и сохраним в anndata objects

### Human

In [12]:
# Define paths and get sample_ids
obj_list = list()
file_list = os.listdir('data/spatial_data/human/')
file_list

['human_759', 'human_j12', 'human_j3', 'human_j4', 'human_j6']

In [14]:
# Read each table and append to our list
for file in file_list:
    print(file)
    ann = pd.read_csv('data/clusters/' + file + '.csv', index_col=0)
    ann.columns = ['label']
    tmp = sc.read_visium('data/spatial_data/human/'+ file)
    tmp.var_names_make_unique()
    tmp.obs['label'] = ann.label
    tmp.obs['sample_id'] = file
    obj_list.append(tmp)

human_759


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


human_j12


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


human_j3


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


human_j4


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


human_j6


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


### Объединим anndata объекты

In [15]:
adata = an.concat( obj_list , merge='same',uns_merge="unique" )
adata.var_names_make_unique()

C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [16]:
adata.uns['spatial'].keys()

dict_keys(['human_759', 'human_j12', 'human_j3', 'human_j4', 'human_j6'])

### Сохраним файлы

In [17]:
os.makedirs('results', exist_ok=True)
os.makedirs('results/spatial_transcriptomics', exist_ok=True)
adata.write_h5ad('results/spatial_transcriptomics/human.h5ad')

### Chimp

In [25]:
# Define paths and get sample_ids
obj_list = list()
file_list = os.listdir('data/spatial_data/chimp/')
file_list

['chimp_11454', 'chimp_13302', 'chimp_13309', 'chimp_j11', 'chimp_j8']

In [26]:
# Read each table and append to our list
for file in file_list:
    print(file)
    ann = pd.read_csv('data/clusters/' + file + '.csv', index_col=0)
    ann.columns = ['label']
    tmp = sc.read_visium('data/spatial_data/chimp/'+ file + '/outs')
    tmp.var_names_make_unique()
    tmp.obs['label'] = ann.label
    tmp.obs['sample_id'] = file
    obj_list.append(tmp)

chimp_11454


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_13302


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_13309


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_j11


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_j8


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


### Объединим anndata объекты

In [27]:
adata = an.concat( obj_list , merge='same',uns_merge="unique" )
adata.var_names_make_unique()

C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [28]:
adata.uns['spatial'].keys()

dict_keys(['11454', 'chimp_13302', 'chimp_13309', 'chimp_j11', 'chimp_j8'])

In [29]:
adata.uns['spatial']['chimp_11454'] = adata.uns['spatial'].pop('11454')

In [30]:
adata.uns['spatial'].keys()

dict_keys(['chimp_13302', 'chimp_13309', 'chimp_j11', 'chimp_j8', 'chimp_11454'])

### Сохраним файлы

In [31]:
os.makedirs('results', exist_ok=True)
os.makedirs('results/spatial_transcriptomics', exist_ok=True)
adata.write_h5ad('results/spatial_transcriptomics/chimp.h5ad')

### Chimp

In [18]:
# Define paths and get sample_ids
obj_list = list()
file_list = os.listdir('data/spatial_data/macaque/')
file_list

['macaque_0701', 'macaque_0703', 'macaque_0704', 'macaque_nb0904']

In [19]:
# Read each table and append to our list
for file in file_list:
    print(file)
    ann = pd.read_csv('data/clusters/' + file + '.csv', index_col=0)
    ann.columns = ['label']
    tmp = sc.read_visium('data/spatial_data/macaque/'+ file)
    tmp.var_names_make_unique()
    tmp.obs['label'] = ann.label
    tmp.obs['sample_id'] = file
    obj_list.append(tmp)

macaque_0701


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


macaque_0703


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


macaque_0704


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


macaque_nb0904


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


### Объединим anndata объекты

In [20]:
adata = an.concat( obj_list , merge='same',uns_merge="unique" )
adata.var_names_make_unique()

C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [21]:
adata.uns['spatial'].keys()

dict_keys(['macaque_0701', 'macaque_0703', 'macaque_0704', 'macaque_nb0904_rotated'])

In [22]:
adata.uns['spatial']['macaque_nb0904'] = adata.uns['spatial'].pop('macaque_nb0904_rotated')

In [23]:
adata.uns['spatial'].keys()

dict_keys(['macaque_0701', 'macaque_0703', 'macaque_0704', 'macaque_nb0904'])

### Сохраним файлы

In [24]:
os.makedirs('results', exist_ok=True)
os.makedirs('results/spatial_transcriptomics', exist_ok=True)
adata.write_h5ad('results/spatial_transcriptomics/macaque.h5ad')